In [248]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk import pos_tag
from nltk import PorterStemmer
import json
import re
from bs4 import BeautifulSoup
import requests
import inflect
import itertools
from tqdm import tqdm
p = inflect.engine()
%matplotlib inline
pd.options.display.max_rows = 999

stop_words = set(stopwords.words('english'))

In [2]:
# Test dataframe with only yummly data

# cuisines = []
# flavors = []
# ingredients = []
# name = []
# source = []
# for filename in os.listdir('data/metadata27638'):
#     if filename.endswith('.json'):
#         data = json.load(open('data/metadata27638/'+filename))
#         cuisines.append(data['attributes']['cuisine'][0])
#         flavors.append(data['flavors'])
#         ingredientStr = ''
#         for ingred in data['ingredientLines']:
#             ingredientStr = ingredientStr + ingred
#         ingredients.append(ingredientStr)
#         name.append(data['name'])
#         source.append(data['source']['sourceDisplayName'])
        
# df = pd.DataFrame({'cuisine':cuisines,'name':name,'flavors':flavors,'ingredients':ingredients,'source':source})

In [3]:
# Reading in data and checking if name and ingredient lists exist

names = []
ingreds = []

for filename in os.listdir('data/recipe_box'):
    if filename.endswith('.json'):
        data = list(json.load(open('data/recipe_box/'+filename)).values())
        for recipe in data:
            if ('title' not in recipe.keys() or 'ingredients' not in recipe.keys()):
                continue
            names.append(recipe['title'])
            ingreds.append(recipe['ingredients'])

In [130]:
# Creating data frame with dish names and ingredients

df = pd.DataFrame({'name':names,'ingredients':ingreds})

In [131]:
df.shape

(124647, 2)

In [138]:
# Remove empty ingredient lists

df = df[df.astype('str')['ingredients']!='[]']
df.shape

(122971, 2)

In [6]:
# Example of ingredient list

df['ingredients'][10]

['3 to 4 cups chicken stock, preferably homemade, recipe follows',
 '1 quart Roasted Winter Vegetables, recipe follows',
 'Kosher salt and freshly ground black pepper',
 '3 (5-pound) roasting chickens',
 '3 large yellow onions, unpeeled, quartered',
 '6 carrots, unpeeled, halved',
 '4 celery stalks with leaves, cut in thirds',
 '4 parsnips, unpeeled, cut in 1/2, optional',
 '20 sprigs fresh parsley',
 '15 sprigs fresh thyme',
 '20 sprigs fresh dill',
 '1 head garlic, unpeeled, cut in 1/2 crosswise',
 '2 tablespoons kosher salt',
 '2 teaspoons whole black peppercorns',
 '1 pound carrots, peeled',
 '1 pound parsnips, peeled',
 '1 large sweet potato, peeled',
 '1 small butternut squash (about 2 pounds), peeled and seeded',
 '3 tablespoons good olive oil',
 '1 1/2 teaspoons kosher salt',
 '1/2 teaspoon freshly ground black pepper',
 '2 tablespoons chopped fresh flat-leaf parsley']

In [139]:
# Example of ingredient list with ADVERTISEMENT tags

df['ingredients'][100000]

['1/2 small head green cabbage, cored and thinly sliced ADVERTISEMENT',
 '1/2 jicama, sliced into matchsticks ADVERTISEMENT',
 '1 large sweet apple (such as Fuji), sliced into matchsticks ADVERTISEMENT',
 '1/2 cup mayonnaise ADVERTISEMENT',
 '1/4 cup pineapple juice ADVERTISEMENT',
 '1 teaspoon white sugar ADVERTISEMENT',
 'hot sauce to taste ADVERTISEMENT',
 'salt and freshly ground black pepper to taste ADVERTISEMENT',
 '1/4 bunch chopped fresh cilantro ADVERTISEMENT',
 '1/3 ounce toasted corn bits (such as CornNuts ®), crushed ADVERTISEMENT',
 'ADVERTISEMENT']

In [176]:
testlist = df['ingredients'][100000]
testlist

['1/2 small head green cabbage, cored and thinly sliced ADVERTISEMENT',
 '1/2 jicama, sliced into matchsticks ADVERTISEMENT',
 '1 large sweet apple (such as Fuji), sliced into matchsticks ADVERTISEMENT',
 '1/2 cup mayonnaise ADVERTISEMENT',
 '1/4 cup pineapple juice ADVERTISEMENT',
 '1 teaspoon white sugar ADVERTISEMENT',
 'hot sauce to taste ADVERTISEMENT',
 'salt and freshly ground black pepper to taste ADVERTISEMENT',
 '1/4 bunch chopped fresh cilantro ADVERTISEMENT',
 '1/3 ounce toasted corn bits (such as CornNuts ®), crushed ADVERTISEMENT',
 'ADVERTISEMENT']

In [177]:
# Scraping corpus of words related to measurements

page = requests.get('https://www.enchantedlearning.com/wordlist/measurement.shtml')
soup = BeautifulSoup(page.content, "html.parser")
measure_corpus = [tag.text for tag in soup.find_all('div',attrs={'class':'wordlist-item'})]
measure_corpus = measure_corpus + [text+'s' for text in measure_corpus] + ['taste','strip', 'strips', 'package' + 'packages' + 'satchet' + 'satchets' + 'sprigs','head','bunch','small','large','big','medium']

In [188]:
def clean_recipe(ingred_list):
    cleanedtext = []
    for ingred in ingred_list:
        # Ignore extra information
        matchtext = re.sub(r' \([^)]*\)', '', ingred)
        
        # Obtain all before first comma
        if re.compile('^(.+?)(?=,)').search(matchtext) is not None:
            matchtext = re.compile('^(.+?)(?=,)').search(matchtext).group(1)
            matchtext = matchtext.replace(' ADVERTISEMENT','')
            
        # Ignore advertisement tag
        elif (ingred == 'ADVERTISEMENT'):
            continue
        
        # If all text, make sure no advertisement tag
        else:
            matchtext = matchtext.replace(' ADVERTISEMENT','')
        
        # Tokenize ingredient list
        tokenized = word_tokenize(matchtext)
        
        # Remove words likely to be stop words or measurements
        removed_stop = [w for w in tokenized if not w in measure_corpus]
        removed_stop = [w for w in removed_stop if not w in stop_words]
        
        # Filter adjectives and nouns
        ingred_words = lambda pos: pos[:2] in ['JJ','NN','NNS']
        ingreds = [word.lower() for (word, pos) in pos_tag(removed_stop) if ingred_words(pos)]
        
        # Convert to singular
        ingreds = [p.singular_noun(word) if p.singular_noun(word) else word for word in ingreds]
        
        # Remove common ingredients 
        common = ['water','salt','pepper']
        cleanedtext.append(ingreds)
        cleanedtext = [[ing for ing in ingreds if not any(word in common for word in ingreds)] for ingreds in cleanedtext]
        
        # Remove additional descriptors for long ingredient names
        cleanedtext = [ingreds[-2:] if len(ingreds) > 2 else ingreds for ingreds in cleanedtext]
   
    return [(' ').join(item) for item in cleanedtext if len(item)>0]

In [192]:
clean_recipe(df['ingredients'][3420])

['cooking spray',
 'bread crumb',
 'olive oil',
 'turkey chicken',
 'pasta',
 'parmesan',
 'white cheddar',
 'cherry tomato',
 'broccoli',
 'asparagu']

In [180]:
#regex test
# re.compile('(?<=\d )(?!.*\d)(.+?)(?=[^a-zA-Z ])').search('1 large sweet apple (such as Fuji), sliced into matchsticks ADVERTISEMENT').group(1)

In [236]:
df['ingredients'][2]

['1 1/2 cups dried black beans, picked over and rinsed',
 '8 cups water, plus 1/4 cup',
 '2 cloves garlic',
 '3 tablespoons vegetable oil',
 '1 large green bell pepper, finely chopped, reserving about 1 teaspoon for garnish',
 '1 small red onion, finely chopped, reserving about 1 teaspoon for garnish',
 '2 teaspoons ground cumin',
 '2 tablespoons cider vinegar',
 'Salt and pepper',
 '1 plum tomato, finely chopped',
 '1 small red chile, finely chopped',
 'Fresh cilantro leaves, for garnish',
 'Dollop sour cream, for garnish',
 'Serving Suggestion: Tortilla chips']

In [195]:
# Clean recipes across dataframe

df['ingreds_list'] = df['ingredients'].apply(clean_recipe)

In [198]:
df['ingreds_list'].head(20)

0     [celery, green onion, parsley, crabmeat, crack...
1     [skirt steak, applewood bacon, red onion, jala...
2     [black bean, clove garlic, vegetable oil, red ...
3     [ground chuck, tomato sauce, egg noodle, sour ...
4     [rice, 1/2-cup quinoa, white tuna, seaweed kel...
5     [olive oil, italian eggplant, hummu hummu, gra...
6     [saffron thread, sugar, orange zest, carrot, u...
7     [hot-dog bun, unsalted butter, lobster lobster...
8     [olive oil, onion, pine nut, white rice, tomat...
9     [cauliflower, clove garlic, unsalted butter, b...
10    [chicken stock, winter vegetable, roasting chi...
11    [chili sauce, rice vinegar, fish sauce, fresh ...
12                        [cheese product, green chily]
13    [london broil, olive oil, worcestershire sauce...
14    [duck, garlic clove, cinnamon stick, fat lard,...
15    [purpose flour, cornmeal, baking powder, sugar...
16    [boneles, egg, bread crumb, deli ham, wine sau...
17    [tortilla chip, neely bbq, pork, bbq sauce

In [202]:
list_ingreds = df['ingreds_list']

In [220]:
combinations = []

for lists in tqdm(list_ingreds):
    combinations = combinations + list(itertools.combinations(lists,2))
    
combinations_df = pd.DataFrame(combinations)

100%|██████████| 122971/122971 [11:20:42<00:00,  3.01it/s]     


In [238]:
combinations_df.columns = ['ingred1','ingred2']
top_ingred1 = combinations_df['ingred1'].value_counts()[:100]
top_ingred2 = combinations_df['ingred2'].value_counts()[:100]

In [255]:
top_ingreds = pd.DataFrame(top_ingred1).join(pd.DataFrame(top_ingred2),how='outer').fillna(0)
top_ingreds['total'] = top_ingreds['ingred1'] + top_ingreds['ingred2']
top_ingreds = top_ingreds.sort_values(by="total", ascending=False)['total']
top_ingreds = top_ingreds.reset_index() 
top_ingreds.columns = ['ingredient','total']

In [256]:
top_ingreds

,ingredient,total
0,olive oil,377461.0
1,sugar,288310.0
2,egg,257308.0
3,all-purpose flour,218468.0
4,clove garlic,216540.0
5,butter,205882.0
6,unsalted butter,184737.0
7,onion,169772.0
8,vanilla extract,142113.0
9,vegetable oil,126008.0


### Categories to rename or combine

sugar + white sugar
clove garlic + garlic clove + garlic
butter + unsalted butter + stick butter
chicken broth + chicken stock
parmesan cheese + parmesan
onion + yellow onion + chopped onion
stalk celery -> celery
remove 'ground'
all-purpose flour + purpose flour
fresh ginger + ginger
slice bacon -> bacon
bay leaf
cilantro leaf + fresh cilantro
parsley leaf + flat-leaf parsley + fresh parsley
basil leaf + fresh basil
mint leaf
thyme leaf + fresh thyme